In [3]:
import pandas as pd
import numpy as np

In [4]:
features = pd.read_csv("/Users/meiramzarypkanov/Desktop/University/4_Network_Security/Network_security/NetworkSecurity/data/ModelData/features.csv")
features.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,166,1,1,0,0,0,0,0.0,0.0,...,32.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
1,60148,83,1,2,0,0,0,0,0.0,0.0,...,32.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
2,123,99947,1,1,48,48,48,48,48.0,0.0,...,40.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
3,123,37017,1,1,48,48,48,48,48.0,0.0,...,32.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
4,0,111161336,147,0,0,0,0,0,0.0,0.0,...,0.0,1753752.625,2123197.578,4822992.0,95.0,9463032.7,2657727.996,13600000.0,5700287.0,BENIGN


In [7]:
features.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [13]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

# Clean column names
features.columns = features.columns.str.replace(' ', '_')

# Sample 5% of data with stratified label distribution
sampled_df, _ = train_test_split(
    features, test_size=0.7, stratify=features['Label'], random_state=42
)

# Prepare data
X = sampled_df.drop(columns=['Label'])
y = LabelEncoder().fit_transform(sampled_df['Label'])

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Randomized search
params = {
    'num_leaves': [31, 50, 70],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

search = RandomizedSearchCV(
    lgb.LGBMClassifier(), params, n_iter=3, scoring='f1_weighted',
    cv=2, n_jobs=4, verbose=0
)
search.fit(X_train, y_train)
final_model = search.best_estimator_

# Evaluate and save
print(classification_report(y_test, final_model.predict(X_test)))

base = '/Users/meiramzarypkanov/Desktop/University/4_Network_Security/Network_security/NetworkSecurity/artifacts/'
joblib.dump(final_model, base + 'lightgbm_network_traffic_model.joblib')
joblib.dump(LabelEncoder().fit(sampled_df['Label']), base + 'label_encoder.joblib')


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033623 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14574
[LightGBM] [Info] Number of data points in the train set: 215822, number of used features: 70
[LightGBM] [Info] Start training from score -0.285414
[LightGBM] [Info] Start training from score -6.818377
[LightGBM] [Info] Start training from score -2.642492
[LightGBM] [Info] Start training from score -2.602741
[LightGBM] [Info] Start training from score -5.789969
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.079852 seconds.
You can set `force_col_wise=true` to remove t

['/Users/meiramzarypkanov/Desktop/University/4_Network_Security/Network_security/NetworkSecurity/artifacts/label_encoder.joblib']